In [1]:
import pdb
import numpy as np
import gcp.bigquery as bq
import gcp.storage as storage
from sklearn.pipeline import Pipeline
try:
   import cPickle as pickle
except:
   import pickle
EST_PICKLE_FILENAME = 'baseline_final_estimator.pkl'

# First feature HAS to be 'district_id' for MAPE calculation.
fields = ['gap', 'district_id', 'timeofday_slot', 'day_in_week', 'sum_price', 'traffic_tj_level1',
            'traffic_tj_level2', 'traffic_tj_level3', 'traffic_tj_level4']
features = fields[1:]

# Scorer Creation (MAPE)

In [ ]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  districts = np.unique(X[:,0])
  district_scores = np.zeros(len(districts))
  for counter, key in enumerate(districts):
    group = np.compress((Xy[:,0] == key).flat, Xy, axis=0)
    district_scores[counter] = np.sum(np.absolute(
        (group[:,-2] -
         group[:,-1])/
        group[:,-2]
      )) / num_timeslots
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  predictions = estimator.predict(X)
  return -mape(X, predictions, y)